project name: flexpart_management
created by diego aliaga daliaga_at_chacaltaya.edu.bo

In [83]:
from useful_scit.imps import *

path = '/homeappl/home/aliagadi/wrk/' \
       'DONOTREMOVE/flexpart_management_data/' \
       'runs/run_2019-10-02_13-42-52_/'
dom = 'd01'
patt = f'*/flxout_{dom}_*.nc'
i_sparse = 100
path_patt = os.path.join(path,patt)

In [84]:
list_files = glob.glob(path_patt)
len(list_files)

21240

In [85]:

files = list_files[0:None:i_sparse]
len(files)

213

In [86]:


ds = xr.open_mfdataset(files,concat_dim='Time',combine='nested')

ds_sum = ds.sum(['ageclass','bottom_top','releases','Time'])


In [ ]:
name = f'sum_rect_{dom}.nc'
dir_path = '/homeappl/home/aliagadi/saltena_2018/flexpart_management/flexpart_management/notebooks/run_2019-10-02_13-42-52_/'
path_out = os.path.join(dir_path,name)
ds_sum.to_netcdf(path_out)

In [ ]:
ds_sum = xr.open_dataset(path_out)

In [ ]:
dc = ds_sum['CONC']

In [ ]:
center = dc.where(dc==dc.max()).to_dataframe().dropna().reset_index().iloc[0]
center

In [ ]:
ds_sum['c_sn'] = dc['south_north']-center['south_north']
ds_sum['c_we'] = dc['west_east']  -center['west_east']

In [ ]:
ds_sum = ds_sum.assign_coords( **{
    'c_sn': ds_sum['c_sn'] ,
    'c_we': ds_sum['c_we']
                        })

In [ ]:
r_dis = \
    (
        ds_sum['c_sn']**2 + ds_sum['c_we']**2
    )**(1/2)

In [ ]:
ds_sum = ds_sum.assign_coords(**{'r_dis':r_dis})

In [ ]:
ds_sc = ds_sum['r_dis']**2 *ds_sum['CONC']

In [ ]:
ds_sc.plot()
ax:plt.Axes=plt.gca()
ax.scatter([center['west_east']],[center['south_north']])